In [1]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
from huggingface_hub import login
login(token='<YOUR TOKEN HERE>')

In [3]:
from transformers import ChameleonProcessor, ChameleonForConditionalGeneration, BitsAndBytesConfig
import torch
from PIL import Image
import requests

In [4]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16,
)

In [5]:
processor = ChameleonProcessor.from_pretrained('facebook/chameleon-7b')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/5.63M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/130 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/102 [00:00<?, ?B/s]

In [6]:
model = ChameleonForConditionalGeneration.from_pretrained('facebook/chameleon-7b',
                                                          quantization_config=quantization_config,
                                                          device_map='cuda:0')

config.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/46.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/204 [00:00<?, ?B/s]

In [7]:
url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

In [8]:
prompt = 'What do you see in this image?<image>'

In [9]:
inputs = processor(images=image,
                   text=prompt,
                   return_tensors='pt').to(model.device, dtype=torch.float16)

In [10]:
inputs

{'input_ids': tensor([[    0, 18105, 16829,  ...,  8711,  8196,  8710]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'), 'pixel_values': tensor([[[[ 0.2324,  0.2559,  0.3259,  ..., -0.0640, -0.1108, -0.0796],
          [ 0.2480,  0.2871,  0.3181,  ..., -0.0328, -0.0328, -0.0484],
          [ 0.2559,  0.2715,  0.2949,  ..., -0.0406, -0.0328,  0.0452],
          ...,
          [ 0.9268,  0.9502,  0.9810,  ...,  0.7314,  0.7158,  0.8174],
          [ 0.9810,  0.9502,  0.9580,  ...,  0.6069,  0.5835,  0.8408],
          [ 0.9580,  0.9424,  0.9736,  ...,  0.5601,  0.7549,  0.7861]],

         [[-0.8208, -0.7974, -0.7349,  ..., -0.8750, -0.9141, -0.8672],
          [-0.8052, -0.7505, -0.7114,  ..., -0.8516, -0.8364, -0.8594],
          [-0.7817, -0.7583, -0.7271,  ..., -0.8750, -0.8516, -0.8052],
          ...,
          [-0.0406, -0.0406, -0.0250,  ..., -0.4385, -0.4463, -0.3682],
          [ 0.0062, -0.0328, -0.0562,  ..., -0.5708, -0.5708, -0.2825]

In [11]:
output = model.generate(**inputs, max_new_tokens=50)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [12]:
output

tensor([[    0, 18105, 16829,  ..., 40733, 16397,     2]], device='cuda:0')

In [13]:
processor.decode(output[0], skip_special_tokens=True)

'What do you see in this image?The image displays two cats sleeping on top of each other on a pink bed. The cat on top has a white collar with a blue tag. The cat under the other has a pink collar.'